# Rhadamanthys
> What is this thing are we just looking at a downloader

- toc: true 
- badges: true
- categories: [rhadamanthys,config,IDA,shifted pointers,PEB,_LIST_ENTRY,_LDR_DATA_TABLE_ENTRY]


## Overview

### Sample

`dca16a0e7bdc4968f1988c2d38db133a0e742edf702c923b4f4a3c2f3bdaacf5` [Malware Bazaar](https://bazaar.abuse.ch/sample/dca16a0e7bdc4968f1988c2d38db133a0e742edf702c923b4f4a3c2f3bdaacf5/)

This malware name is crazy, we will refer to it as "ramen noodls" for simplicity.

### References
- [Triage Run](https://tria.ge/221226-1ep5eadg74/behavioral2#report)
- [Dancing With Shellcodes: Analyzing Rhadamanthys Stealer](https://elis531989.medium.com/dancing-with-shellcodes-analyzing-rhadamanthys-stealer-3c4986966a88)
  - Downloader `af04ee03d69a7962fa5350d0df00fafc4ae85a07dff32f99f0d8d63900a47466`
 

## Stage 1
The first stage is a C++ loader that is used to decrypt, load, and execute the 2nd stage shellcode. The program flow of the loader is difficult to follow due to the C++ and some nested structures and callbacks. It is not clear if this was intentional or note. 

The shellcode appears to be located in what appears to be a giant Base64 encoded string (not confirmed). This first stage may have alos implmented some PAGE_GUARD exception handling progrem flow redirection for anti-analysis (not confrimed). Dispite this it is trivial to execute the stage until it jumps to the decrypted loaded shellcode (as long as the PAGE_GUARD exceptions are passed to the process).

## Stage 2

For analysis we just dumped the entire memory region that contained the second stage shellcode. The region is based at `0x00760000` but the shellcode entry is at `0x00780A68`. The memory region dump is available on Malshare [`d4f37699c4b283418d1c73416436826e95858cf07f3c29e6af76e91db98e0fc0`](https://malshare.com/sample.php?action=detail&hash=d4f37699c4b283418d1c73416436826e95858cf07f3c29e6af76e91db98e0fc0).

The first task of the shellcode is to walk the PEB to locate Kernel32.

### PEB Walk _LDR_DATA_TABLE_ENTRY and Shifted Pointers in IDA

The process of "walking the PEB" to access the modules loaded in a process as been a shellcode meme since the beginning of shellcode. It's everywhere, and we all mostly understand how it works. 

The issue comes when we try to represent this cleanly in IDA's pseudocode view... accessing a `_LDR_DATA_TABLE_ENTRY` via its `LIST_ENTRY` in the `_PEB_LDR_DATA` structure creates a very messy IDB. Instead of the `_LDR_DATA_TABLE_ENTRY` members we see unhelpful offsets relative to the Flink and Blink member of the `LIST_ENTRY`.

![](https://i.imgur.com/Ta8hGnm.png)

The reason this happens is that the `InMemoryOrderModuleList` is defined as a `LIST_ENTRY` in the `_PEB_LDR_DATA`. The `LIST_ENTRY` struct simply describes a linked list as follows.

```C
typedef struct _LIST_ENTRY {
   struct _LIST_ENTRY *Flink;
   struct _LIST_ENTRY *Blink;
} 
```

The problem comes from where that struct is actually located in the `_LDR_DATA_TABLE_ENTRY`...

```C
typedef struct _LDR_DATA_TABLE_ENTRY {
    PVOID Reserved1[2];
    LIST_ENTRY InMemoryOrderLinks; //<---- Offset into the struct!
    PVOID Reserved2[2];
    PVOID DllBase;
    PVOID EntryPoint;
    PVOID Reserved3;
    UNICODE_STRING FullDllName;
    BYTE Reserved4[8];
    PVOID Reserved5[3];
    union {
        ULONG CheckSum;
        PVOID Reserved6;
    };
    ULONG TimeDateStamp;
}
```

Because the `LIST_ENTRY` is offset into the `_LDR_DATA_TABLE_ENTRY` it means that when we attempt to cast a pointer to a `LIST_ENTRY` as a `_LDR_DATA_TABLE_ENTRY` we are off by some amount (in this case 2 * PVOID = 8 bytes). The following diagram attempts to explain the issue.

![](https://i.imgur.com/1G44FsV.png)

One hack might be to create our own custom struct that starts at the offset, but this would be madness when dealing with more than a few types... instead we have... **shifted pointers**!

#### IDA Shifted Pointers
IDA has a simple concept (with some insane syntax) to deal with this issue called a [shifted pointer](https://www.hex-rays.com/products/ida/support/idadoc/1695.shtml). When assigning a type to `LIST_ENTRY` the shifted pointer syntax can be used to tell IDA that it is actually a pointer inside a larger struct with an offset. 

```C
_LIST_ENTRY *__shifted(_LDR_DATA_TABLE_ENTRY,8) pListEntry
```

![](https://i.imgur.com/hvVJkP1.png)

#### References 
- PE Header notes [yates](https://web.archive.org/web/20181222200541/http://www.woodmann.com/yates/PE_Information/PE_Notes.pdf)
- Geoff Chappell [LDR_DATA_TABLE_ENTRY](https://www.geoffchappell.com/studies/windows/km/ntoskrnl/inc/api/ntldr/ldr_data_table_entry.htm)
- [Understanding LIST_ENTRY Lists and Its Importance in Operating Systems] (https://www.codeproject.com/Articles/800404/Understanding-LIST-ENTRY-Lists-and-Its-Importance)
- MSDN [PEB_LDR_DATA](https://learn.microsoft.com/en-us/windows/win32/api/winternl/ns-winternl-peb_ldr_data)
- IDA Tricks [CONTAINING_RECORD macro](https://www.hex-rays.com/products/decompiler/manual/tricks.shtml#03)
- IDA docs [Shifted Pointers](https://www.hex-rays.com/products/ida/support/idadoc/1695.shtml)

#### Thanks 
- [@yates82](https://twitter.com/yates82)
- [@printup](https://github.com/anthonyprintup)
- [@dodo](https://twitter.com/dodo_sec)

Thanks to everyone who helped me figure this out for once and for all! 



## Analysis

This stage only serves one purpose; unpack and execute the final stage. The unpacking algorithm is currently unknown!


### Thanks Mishap
[Full analysis of Stage 2 (with unpacker)](https://gist.github.com/oopsmishap/91f41810e41cecec7d7d3760bbc4ba67)

## Emulation Attempt

Instead of analyzing this intermediate stage we are going to try and emulate passed it.

In [35]:
from dumpulator import Dumpulator

dp = Dumpulator("/tmp/stage2.dmp", quiet=True)
shellcode_start = dp.regs.eip

print(hex(shellcode_start))


shell_base = 0x00810000
shell_end = 0x0082CFF5

0x82cc90


In [36]:
dp.start(shellcode_start, end=shell_end)

commit(0x84a000[0x1d000], PAGE_READWRITE)
commit(0x21d0000[0x1d000], PAGE_EXECUTE_READWRITE)


In [37]:
stage3_start = dp.regs.ebx

print(hex(stage3_start))

0x21d607f


In [38]:
shell_page = dp.memory.find_region(stage3_start)
print(shell_page.size)
print(shell_page.start)
print(f"Shellcode entrypoint: {hex(stage3_start - shell_page.start)}")
print(f"Shellcode base: {hex(shell_page.start)}")
shell_page_data = dp.read(shell_page.start, shell_page.size)
open('/tmp/dump_stage3.bin','wb').write(shell_page_data)

118784
35454976
Shellcode entrypoint: 0x607f
Shellcode base: 0x21d0000


118784

We don't yet have a way to add exception hooks to dumpulator so if we want to do this without knowing the address of the end of **stage 2** then we need to modify the dumpulator source code. 

In [39]:
print(hex(dp.read_long(dp.regs.esp)))
print(hex(dp.read_long(dp.regs.esp-4)))

0x21d0000
0x19fefc


In [40]:
dp.memory.find_region(0x19fefc)

MemoryRegion(0x1a0000, 0x4000, PAGE_READONLY, MemoryType.MEM_MAPPED, None)

In [41]:
dp.exports.get(0x7656f530)

'kernel32.dll:LocalFree'

In [42]:
#dp.print_memory()
hex(dp.read_long(0x21d0000))

'0x14c5352'

In [23]:
dp.read(0x13BAC, 10)

bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [43]:
dp.start(dp.regs.eip, end=0x021D5D88 )

In [45]:
dp.read(0x13BAC, 100)

bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [26]:
dp.start(dp.regs.eip, end=0x021D6098)


In [34]:
dp.modules.find(dp.regs.eax)

Module(0x76550000, 0xf0000, 'C:\\Windows\\System32\\kernel32.dll')

In [46]:
dp = Dumpulator("/tmp/stage2.dmp", quiet=True)
shellcode_start = dp.regs.eip

print(hex(shellcode_start))


shell_base = 0x00810000
shell_end = 0x0082CFF5
dp.start(shellcode_start, end=shell_end)
print(f"EIP: {hex(dp.regs.eip)}")
dp.start(dp.regs.eip, end=0x021D5D88)
dp.regs.esi

0x82cc90
commit(0x84a000[0x1d000], PAGE_READWRITE)
commit(0x21d0000[0x1d000], PAGE_EXECUTE_READWRITE)
EIP: 0x82cff5


35535788

In [53]:
print(hex(dp.regs.esi))
dp.read(dp.regs.esi, 100)

0x21e3bac


bytearray(b'k\x00e\x00r\x00n\x00e\x00l\x003\x002\x00.\x00d\x00l\x00l\x00\x00\x00\x00\x00ZwQueryInformationProcess\x00\x00\x00ntdll.dll\x00\x00\x00KiUserExceptionDispatcher\x00\x00\x00,\x00\x00\x00')

8454144

#### Rename IAT Hashes in IDA
```python
### apis = {dict with hashdb enum of all imports}

api_nums = dict((v,k) for k,v in apis.items())

ptr = 0x0001B114

while ptr < 0x0001BC9E:
    api_hash = ida_bytes.get_dword(ptr)
    api_name = api_nums.get(api_hash,'')
    if api_name != '':
        print("ptr_"+api_name)
        idc.set_name(ptr, "ptr_"+api_name, 0x800) #SN_FORCE
        ptr += 4
    else:
        ptr += 1
    
```